In [55]:
import pandas as pd

In [56]:
conv_factors = pd.read_csv('/Users/lukecullen/Library/CloudStorage/OneDrive-UniversityofCambridge/PhD/C-THRU/zenodo_data/processConversionFactors_allgases_allalloc_dedoubled.csv')
conv_factors.drop(columns=[i for i in conv_factors.columns if 'Electricity' in i], inplace=True)
conv_factors.rename(columns={'ihs_match':'Process'}, inplace=True)
conv_factors.fillna(0, inplace=True)

In [66]:
gases = ['CO2e_100a', 'CO2e_20a', 'Carbon dioxide', 'Carbon monoxide', 'Chloroform', 'Dinitrogen monoxide', 'Ethane', 'Methane', 'Nitric oxide', 'Nitrogen fluoride', 'Perfluoropentane', 'Sulfur hexafluoride', 'Other']
names = ['CO2e_100a', 'CO2e_20a', 'CO2', 'CO', 'CHCl3', 'NOx', 'C2H6', 'CH4', 'N2O', 'NF3', 'C5F12', 'SF6', 'Other']

for allocation in ['energy']:
    for gas, name in zip(gases, names):

        df = conv_factors[['Product', 'Process']+[i for i in conv_factors.columns if (allocation in i) and (gas in i)]]
        
        initial = [', '+allocation+' allocation', ' factor', ' sigma', ' '+gas, 'ihs_cradle-to-out-gate']
        replacement = ['', '', ' uncertainty','','Total']
        
        for i in range(len(initial)):
            df.columns = df.columns.str.replace(initial[i], replacement[i])
        
        df['Intermediates'] = df['Organic chemicals'].values# + df['Other intermediates'].values
        df['Intermediates uncertainty'] = df['Organic chemicals uncertainty'].values# + df['Other intermediates uncertainty'].values
        df.drop(columns=['Organic chemicals', 'Organic chemicals uncertainty'], inplace=True)
        # df['Intermediates'] = df['Organic chemicals'].values + df['Other intermediates'].values
        # df['Intermediates uncertainty'] = df['Organic chemicals uncertainty'].values + df['Other intermediates uncertainty'].values
        # df.drop(columns=['Organic chemicals', 'Other intermediates', 'Organic chemicals uncertainty', 'Other intermediates uncertainty'], inplace=True)
        
        df.to_csv('/Users/lukecullen/Library/CloudStorage/OneDrive-UniversityofCambridge/PhD/C-THRU/zenodo_data/processEFs_'+name+'_'+allocation+'Allocation.csv', index=False)

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_4075/1075175095.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Intermediates'] = df['Organic chemicals'].values# + df['Other intermediates'].values
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_4075/1075175095.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Intermediates uncertainty'] = df['Organic chemicals uncertainty'].values# + df['Other intermediates uncertainty'].values
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h00

In [114]:
## Per country
country = pd.read_csv('/Users/lukecullen/Library/CloudStorage/OneDrive-UniversityofCambridge/PhD/C-THRU/zenodo_data/requested_outputs/ef_breakdown_by_type_CO2e_20a.csv').fillna(0)
country.columns = country.columns.str.replace('_sigma', ' uncertainty')

In [115]:
# Add together Organic chemicals and Other intermediates rows for each ['Product', 'COUNTRY/TERRITORY'] pairing into a row called Intermediates
ints = country[[i in ['Organic chemicals', 'Other intermediates'] for i in country['Type']]].groupby(['PRODUCT', 'COUNTRY/TERRITORY']).sum().reset_index()
ints['Type'] = 'Intermediates'
country_new = pd.concat((country[[i not in ['Organic chemicals', 'Other intermediates'] for i in country['Type']]], ints))
totals = country_new.groupby(['PRODUCT', 'COUNTRY/TERRITORY']).sum().reset_index()
totals['Type'] = 'Total'
country_new = pd.concat((country_new, totals))
country_new.sort_values(by=['PRODUCT', 'COUNTRY/TERRITORY', 'Type'], inplace=True)

In [116]:
country_new.to_csv('/Users/lukecullen/Library/CloudStorage/OneDrive-UniversityofCambridge/PhD/C-THRU/zenodo_data/countryEF_CO2e_20a.csv', index=False)

In [109]:
country_new

,PRODUCT,COUNTRY/TERRITORY,Type,1978,1979,1980,1981,1982,1983,1984,...,2041 uncertainty,2042 uncertainty,2043 uncertainty,2044 uncertainty,2045 uncertainty,2046 uncertainty,2047 uncertainty,2048 uncertainty,2049 uncertainty,2050 uncertainty
0,"1,4-BUTANEDIOL",BELGIUM,Direct Process,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,"1,4-BUTANEDIOL",BELGIUM,Direct Utilities,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,"1,4-BUTANEDIOL",BELGIUM,Feedstock,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,"1,4-BUTANEDIOL",BELGIUM,Indirect Utilities,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0,"1,4-BUTANEDIOL",BELGIUM,Intermediates,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24747,VINYL ACETATE M.,UNITED STATES,Feedstock,3.996712e-13,3.021904e-13,2.890955e-13,2.890955e-13,1.487627e-13,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
24748,VINYL ACETATE M.,UNITED STATES,Indirect Utilities,2.996793e-11,2.993765e-11,2.993358e-11,2.993358e-11,2.988999e-11,2.984378e-11,2.984378e-11,...,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13,4.476567e-13
3535,VINYL ACETATE M.,UNITED STATES,Intermediates,8.486444e-11,8.492305e-11,8.493092e-11,8.493092e-11,8.501530e-11,8.510475e-11,8.510475e-11,...,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12,1.276571e-12
24751,VINYL ACETATE M.,UNITED STATES,Primary chemicals,6.556218e-14,6.692139e-14,6.710398e-14,6.710398e-14,6.906070e-14,7.113496e-14,7.113496e-14,...,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15,1.067024e-15
